In [1]:
from chi import server, context, lease
import os, time

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@TACC")

In [2]:
l = lease.get_lease(f"mlops-dev-project23")
l.show()

HTML(value='\n        <h2>Lease Details</h2>\n        <table>\n            <tr><th>Name</th><td>mlops-dev-proj…

Lease Details:
Name: mlops-dev-project23
ID: 28bac268-715d-4b99-b642-3812cfeb3bb9
Status: ACTIVE
Start Date: 2025-05-10 12:00:00
End Date: 2025-05-11 04:00:00
User ID: f9a642531ef78b49697f8beaba43bd4081bc2584478a0f280c0577547b859bc6
Project ID: d3c6e101843a4ba79e665ebf59b521a2

Node Reservations:
ID: 5a14403c-15f1-4afa-b739-0085bc1fc05f, Status: active, Min: 1, Max: 1

Floating IP Reservations:

Network Reservations:

Events:


In [4]:
username = "project23"
s = server.Server(
    f"node-dcntrain-{username}", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA",
    key_name="id_rsa_project23"
)
s.submit(idempotent=True)

Waiting for server node-dcntrain-project23's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-dcntrain-project23
Id,8191aa31-8915-4b94-8d54-4cd41349fc6f
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.3.20 (v4) Type: fixed MAC: 44:a8:42:26:f8:62
Network Name,None
Created At,2025-05-10T14:11:24Z
Keypair,id_rsa_project23
Reservation Id,None
Host Id,9acf860df16fe3cd915f9522cd52cf171577a815ef5c486f67a143e3


In [5]:
s.associate_floating_ip()

In [6]:
s.refresh()
s.check_connectivity()
# ssh -i ~/.ssh/id_rsa_project23 cc@129.114.109.122

Checking connectivity to 129.114.109.122 port 22.


Connection successful


In [7]:
s.refresh()
s.show(type="widget")

Attribute,node-dcntrain-project23
Id,8191aa31-8915-4b94-8d54-4cd41349fc6f
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.3.20 (v4) Type: fixed MAC: 44:a8:42:26:f8:62 IP: 129.114.109.122 (v4) Type: floating MAC: 44:a8:42:26:f8:62
Network Name,sharednet1
Created At,2025-05-10T14:11:24Z
Keypair,id_rsa_project23
Reservation Id,5a14403c-15f1-4afa-b739-0085bc1fc05f
Host Id,9acf860df16fe3cd915f9522cd52cf171577a815ef5c486f67a143e3


## Setting up Docker

In [8]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.109.122: b'dc2ec3583d37eceaa729b42b5c870ed9'
  warnings.warn(


# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c doc

Client: Docker Engine - Community
 Version:           28.1.1
 API version:       1.49
 Go version:        go1.23.8
 Git commit:        4eba377
 Built:             Fri Apr 18 09:52:14 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.1.1
  API version:      1.49 (minimum version 1.24)
  Go version:       go1.23.8
  Git commit:       01f442b
  Built:            Fri Apr 18 09:52:14 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>

## Set up NVIDIA GPU

If the chosen OS image doesn't include CUDA, you'll need to install the drivers for the NVIDIA GPUs yourself with the following commands

In [ ]:
# Only run if not using a CUDA OS image
s.execute("sudo apt update")
s.execute("sudo apt install ubuntu-drivers-common")
s.execute("sudo ubunti-drivers autoinstall")
s.execute("sudo reboot")

After we've installed the drivers, we can move on to installing the NVIDIA CUDA toolkit so we can actually use the GPUs

In [9]:
s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
s.execute("sudo apt update")
s.execute("sudo apt-get install -y nvidia-container-toolkit")
s.execute("sudo nvidia-ctk runtime configure --runtime=docker")
# for https://github.com/NVIDIA/nvidia-container-toolkit/issues/48
s.execute("sudo jq 'if has(\"exec-opts\") then . else . + {\"exec-opts\": [\"native.cgroupdriver=cgroupfs\"]} end' /etc/docker/daemon.json | sudo tee /etc/docker/daemon.json.tmp > /dev/null && sudo mv /etc/docker/daemon.json.tmp /etc/docker/daemon.json")
s.execute("sudo systemctl restart docker")

deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/stable/deb/$(ARCH) /
#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/experimental/deb/$(ARCH) /


Get:1 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease [1477 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Hit:3 https://download.docker.com/linux/ubuntu noble InRelease
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:5 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:6 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  Packages [18.6 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Fetched 528 kB in 1s (922 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
4 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libnvidia-container-tools libnvidia-co

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 5849 kB in 2s (3532 kB/s)
Selecting previously unselected package libnvidia-container1:amd64.
(Reading database ... 113813 files and directories currently installed.)
Preparing to unpack .../libnvidia-container1_1.17.6-1_amd64.deb ...
Unpacking libnvidia-container1:amd64 (1.17.6-1) ...
Selecting previously unselected package libnvidia-container-tools.
Preparing to unpack .../libnvidia-container-tools_1.17.6-1_amd64.deb ...
Unpacking libnvidia-container-tools (1.17.6-1) ...
Selecting previously unselected package nvidia-container-toolkit-base.
Preparing to unpack .../nvidia-container-toolkit-base_1.17.6-1_amd64.deb ...
Unpacking nvidia-container-toolkit-base (1.17.6-1) ...
Selecting previously unselected package nvidia-container-toolkit.
Preparing to unpack .../nvidia-container-toolkit_1.17.6-1_amd64.deb ...
Unpacking nvidia-container-toolkit (1.17.6-1) ...
Setting up nvidia-container-toolkit-base (1.17.6-1) ...
Setting up libnvidia-container1:amd64 (1.17.6-1) ...
Setting up lib

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
time="2025-05-10T14:29:27Z" level=info msg="Config file does not exist; using empty config"
time="2025-05-10T14:29:27Z" level=info msg="Wrote updated config to /etc/docker/daemon.json"
time="2025-05-10T14:29:27Z" level=info msg="It is recommended that docker daemon be restarted."


<Result cmd='sudo systemctl restart docker' exited=0>

Verify that we can access the GPUs from within a container

In [10]:
s.execute("docker run --rm --gpus all ubuntu nvidia-smi")

Unable to find image 'ubuntu:latest' locally
latest: Pulling from library/ubuntu
0622fac788ed: Pulling fs layer
0622fac788ed: Verifying Checksum
0622fac788ed: Download complete
0622fac788ed: Pull complete
Digest: sha256:6015f66923d7afbc53558d7ccffd325d43b4e249f41a6e93eef074c9505d2233
Status: Downloaded newer image for ubuntu:latest


Sat May 10 14:29:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-PCIE-16GB           Off |   00000000:03:00.0 Off |                    0 |
| N/A   31C    P0             28W /  250W |       1MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

<Result cmd='docker run --rm --gpus all ubuntu nvidia-smi' exited=0>

install nvtop so that we can monitor the GPU usage

In [11]:
s.execute("sudo apt update")
s.execute("sudo apt -y install nvtop")

Hit:1 https://download.docker.com/linux/ubuntu noble InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Hit:3 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:5 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:6 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Fetched 256 kB in 1s (407 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
4 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  nvtop
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 62.8 kB of archives.
After this operation, 180 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble/multiverse amd64 nvtop amd64 3.0.2-1 [62.8 kB]


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 62.8 kB in 1s (122 kB/s)
Selecting previously unselected package nvtop.
(Reading database ... 113837 files and directories currently installed.)
Preparing to unpack .../nvtop_3.0.2-1_amd64.deb ...
Unpacking nvtop (3.0.2-1) ...
Setting up nvtop (3.0.2-1) ...
Processing triggers for man-db (2.12.0-4build2) ...


debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.


<Result cmd='sudo apt -y install nvtop' exited=0>

## Mount the object store onto the node

In [ ]:
s.execute("curl https://rclone.org/install.sh | sudo bash")
s.execute("sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf")
s.execute("mkdir -p ~/.config/rclone")
s.execute('''cat > ~/.config/rclone/rclone.conf <<'EOF'
[chi_tacc]
type = swift
user_id = 59b49300deb6f11832f7764d9f2c3451ba51e578bb556d58e02bf4c64bd89a31
application_credential_id = aba2c756cb1649b6bd47c572f67a9528
application_credential_secret = HnUZC1Dvhnw1A5-19LiDQORiWURpL1IflqK3tk506NXJp8Tb0HSW4m0V9Ch4zEhwV9WR7tTxiAPTcTGHMi2SKw
auth = https://chi.tacc.chameleoncloud.org:5000/v3
region = CHI@TACC
EOF''')
s.execute("rclone lsd chi_tacc:")
s.execute("export RCLONE_CONTAINER=object-persist-project23")
s.execute("sudo mkdir -p /mnt/object")
s.execute("sudo chown -R cc /mnt/object")
s.execute("sudo chgrp -R cc /mnt/object")
s.execute("rclone mount chi_tacc:object-persist-project23 /mnt/object --read-only --allow-other --daemon")
s.execute("ls /mnt/object")

    curl https://rclone.org/install.sh | sudo bash

    sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf

    mkdir -p ~/.config/rclone

    # Write your rclone.conf
    cat > ~/.config/rclone/rclone.conf <<'EOF'
    [chi_tacc]
    type = swift
    user_id = your_user_id
    application_credential_id = your_application_id
    application_credential_secret = your_application_secret
    auth = https://chi.tacc.chameleoncloud.org:5000/v3
    region = CHI@TACC
    EOF

    rclone lsd chi_tacc:

    export RCLONE_CONTAINER=object-persist-project23

    sudo mkdir -p /mnt/object
    sudo chown -R cc /mnt/object
    sudo chgrp -R cc /mnt/object

    rclone mount chi_tacc:object-persist-project23 /mnt/object --read-only --allow-other --daemon

    ls /mnt/object

## Pull and start container for a single GPU

In [12]:
s.execute("docker pull quay.io/jupyter/pytorch-notebook:cuda12-pytorch-2.5.1")

cuda12-pytorch-2.5.1: Pulling from jupyter/pytorch-notebook
54609b48ebc1: Pulling fs layer
1bf84e16ee78: Pulling fs layer
70c528583d48: Pulling fs layer
4f4fb700ef54: Pulling fs layer
d20e2bbc3444: Pulling fs layer
e1236ab05074: Pulling fs layer
eb55310ee8e5: Pulling fs layer
247e1eb593d5: Pulling fs layer
1ac7d388a98d: Pulling fs layer
f27656f139fb: Pulling fs layer
d20e2bbc3444: Waiting
ac0c7c38d6ad: Pulling fs layer
c48c65e32957: Pulling fs layer
e1236ab05074: Waiting
d79a028c87d5: Pulling fs layer
247e1eb593d5: Waiting
97452dad12c2: Pulling fs layer
eb55310ee8e5: Waiting
1ac7d388a98d: Waiting
aad099df5a3e: Pulling fs layer
41e13f849bef: Pulling fs layer
f27656f139fb: Waiting
3734b2c4ea95: Pulling fs layer
5d39b32f56c0: Pulling fs layer
ac0c7c38d6ad: Waiting
c48c65e32957: Waiting
4f4fb700ef54: Waiting
d79a028c87d5: Waiting
dfe267ba30ed: Pulling fs layer
3734b2c4ea95: Waiting
5d39b32f56c0: Waiting
aad099df5a3e: Waiting
92e65b90a905: Pulling fs layer
bc6b6634a9a2: Pulling fs layer
dfe

<Result cmd='docker pull quay.io/jupyter/pytorch-notebook:cuda12-pytorch-2.5.1' exited=0>

s.execute("""
docker run -it --rm -p 8888:8888 \
--gpus all \
--name torchnb \
-v ~/my_workspace:/home/jovyan/work \
-v /mnt/object:/home/jovyan/work/data \
-u 1000:1000 \
quay.io/jupyter/pytorch-notebook:cuda12-latest
""")
